# Dublin Bike map creation


In [ ]:
try:
    import time
    import getpass
    import requests
    from arcgis.gis import GIS
    from arcgis.features import FeatureLayerCollection
    from arcgis.mapping import WebMap
    
    print('Modules loaded successfully')
    
except ImportError as e:
    print(f"Error importing module: {e}")

In [ ]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

    """
    Prompt the user to log in to ArcGIS Online (AGOL) by entering their username and password.
    
    This function uses the `input` and `getpass` modules to securely prompt the user for their
    ArcGIS Online credentials, creates a GIS object, and attempts to log in to the AGOL portal.
    
    Returns:
        GIS: A `GIS` object that represents the authenticated session with ArcGIS Online if 
        the login is successful.
        None: If the login fails (e.g., due to incorrect credentials or an error during the 
        login process), the function returns `None`.

    Raises:
        Exception: Any exception encountered during the login process is caught and printed to
        the console, with the function returning `None`.
    """ 

def login_AGOL():

    try:
        # prompt the user to enter their AGOL username
        username = input("Enter your ArcGIS Online username:")

        # prompt the user to enter their AGOL password (without echoing)
        password = getpass.getpass("Enter your ArcGIS Online password:")

        # create a GIS object and login to AGOL
        gis = GIS("https://www.arcgis.com", username, password)

        # confirm the login was successful
        if gis.users.me is not None:
            print ("Login successful. Logged in as:", gis.users.me.username)
        else:
            print ("Log in failed. Please check credientials.")
            return None
    
        # return the object
        return gis
    
    except Exception as e:
        print ("An error occurred during login:", e)
        return None
    
# run the login function

gis = login_AGOL()

In [ ]:
# Function to publish GeoJSON data to AGOL
    
    """
    Publishes GeoJSON data related to Dublin Bike Stations and Zones to ArcGIS Online (AGOL).

    This function checks if a feature layer titled "BikeStations_Zones" already exists on AGOL. 
    If it does not exist, the function publishes a new feature layer using the provided GeoJSON data.
    If it already exists, the function retrieves the existing layer.

    Returns:
        FeatureLayer: A `FeatureLayer` object if the GeoJSON data is successfully published 
        or retrieved. 
        None: If an error occurs during the process, the function returns `None`.

    Raises:
        Exception: Catches and prints any exception encountered during the publishing process.
    """

def publish_geojson_to_agol(gis):
       
    try:
        BikeStationsZone_data = None

        # Search for existing feature layers with the title "BikeStations_Zones"
        zoneSearch = gis.content.search("title:BikeStations_Zones", item_type="Feature Layer")

        # If no existing feature layer is found, create a new one
        if not zoneSearch:
            # Set up the feature service properties in AGOL
            item_properties = {
                "title": "BikeStations_Zones",
                "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes",
                "tags": "dublinBikes",
                "type": "GeoJson"  # Note: Ensure this is the correct type, usually 'Feature Layer' is used for publishing
            }

            # URL link to the GeoJSON file
            geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
            geojson_item = gis.content.add(item_properties, geojson_url)

            # Publish the GeoJSON as a feature layer
            BikeStationsZone_data = geojson_item.publish()
            print("Data published. New feature layer created.")
            
        else:
            # Handle the case where the feature layer already exists
            BikeStationsZone_data = zoneSearch[0]
            print("Bike Station and Zone data already exists in AGOL.")
        
        return BikeStationsZone_data

    except Exception as e:
        print("An error occurred during publishing:", e)
        return None

# publish the geojson
BikeStationsZone_data = publish_geojson_to_agol(gis)

In [ ]:
# make a request to the rest endpoint and verify the response

    """
    Fetches bike data from the specified API endpoint with retry logic.

    This function attempts to retrieve data from a given URL, handling potential 
    failures by retrying a specified number of times with a delay between attempts.
    It parses the JSON response if the request is successful and returns the data.
    If all retry attempts fail, it returns `None`.

    Args:
        url (str): The URL of the API endpoint to fetch the bike data from.
        max_retries (int): The maximum number of retry attempts if the 
                                      request fails. Default is 5.
        retry_delay (int): The number of seconds to wait between retry 
                                      attempts. Default is 60.

    Returns:
        dict or None: The parsed JSON data from the API if successful, or `None` 
                      if all retry attempts fail.

    Raises:
        requests.RequestException: If a network-related error occurs, the function
                                  will handle it and retry according to the 
                                  retry logic.
    """

# URL to API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/bikes/bleeper_bikes/current/bikes.geojson"

def fetch_bike_data(url, max_retries=5, retry_delay=60):
 
    attempt = 0
    while attempt < max_retries:
        try:
            response = requests.get(url)
            if response.status_code in [200, 201]:
                # Parse the JSON response
                bike_data = response.json()
                return bike_data  # Data retrieval successful
            else:
                print(f'Failed to retrieve data. Status code: {response.status_code}. Retrying in {retry_delay} seconds...')
        except requests.RequestException as e:
            print(f'An error occurred: {e}. Retrying in {retry_delay} seconds...')
        
        # Increment attempt counter and wait before retrying
        attempt += 1
        time.sleep(retry_delay)

    return None  # Data retrieval failed after retries

# Fetch the bike data
bike_data = fetch_bike_data(bike_url)

# Output message based on data retrieval success or failure
if bike_data:
    print('Data retrieved successfully.')
else:
    print('No data retrieved, cannot proceed with further actions.')

In [ ]:
# create a FLC from the GeoJSON

    """
    Manages the creation and updating of a Feature Layer Collection (FLC) for live 
    Dublin Bike data in ArcGIS Online (AGOL) using GeoJSON data.

    This script performs the following tasks:
    1. Checks if a Feature Layer with a specific title already exists in AGOL.
    2. If the Feature Layer does not exist, it uploads a GeoJSON file and publishes 
    it as a new Feature Service.
    3. If the Feature Layer already exists, it overwrites the existing Feature Layer 
    Collection with new GeoJSON data.

    Args:
        item_properties (dict): A dictionary containing properties for the item to be 
        added to AGOL. Must include:
            - "title" (str): The title of the item.
            - "description" (str): A description of the data.
            - "tags" (str): Tags for improving search functionality.
            - "type" (str): The type of data ("GeoJson" in this case).
        bike_url (str): URL of the GeoJSON file to be uploaded and published.

    Returns:
        FeatureLayerCollection: A "FeatureLayerCollection" object published if successful.

    Exceptions:
        - Raises and handles exceptions related to uploading and publishing data.
        - Prints messages to indicate success or failure of operations.
    """

# set up the feature service properties in AGOL for the stations and zone dataset
item_properties = {
    "title": "Bike_Locations", #provide a title for your item
    "description": "Dublin Bike current locations", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }
    
# search AGOL contents to check if the layer already exists
fl_search = gis.content.search("title:Bike_Locations",item_type="Feature Layer")

if not fl_search:
    # if search list is empty (meaning a feature layer with that title does not exist), upload the GeoJSON 
    # & publish as a new feature service
    try:
        geojson_item = gis.content.add(item_properties, bike_url)
        live_bike_fl = geojson_item.publish()
        
        print("Successfully created new feature service and published live bike location")
   
    except Exception as e:
        if "already exists" in str(e):
            print("Bike_Locations has been created")
        else:
            print(f"An error occurred during the process: {e}")    
    
   # geojson_item = gis.content.add(item_properties, bike_url)
   # live_bike_fl = geojson_item.publish()
   # print("Successfully created new feature service and published live bike location")
    
else: 
    # if search list is NOT empty (meaning feature layer with that title already exists), overwrite the data within the FL
    live_bike_fl = fl_search[0]
    content = FeatureLayerCollection.fromitem(live_bike_fl)
    
    try:
        # overwriting the FLC data with new data
        overwrite_result = content.manager.overwrite(bike_url)

        # overwrite_result will have {"success":True} or {"success":False} - check the result
        if not overwrite_result['success']:
            print("Failed to overwrite Feature Layer Collection with new GeoJSON Data")
        else:
            print("Successfully updated Feature Layer Collection with new GeoJSON Data")
        
    except Exception as e:
        print("An error occurred during the process: {e}")

In [ ]:
#  Search AGOL content to see if a map has already been created

    """
    Searches for an existing web map in ArcGIS Online (AGOL) with a specified title.

    This code checks AGOL content to see if a web map using a given title already exists. 
    If a map with the specified title is found, it retrieves the web map item and creates 
    a `WebMap` object from it. If no map with the title is found, it prints a message 
    indicating that no existing map was found.

    Args:
        map_title (str): The title of the web map to search for in AGOL.

    Returns:
        Webmap: A `WebMap` object if a web map with the specified title is found in AGOL.
    """

# the title we are searching for
map_title = "DublinBike_LocationMap"

# searching in AGOL for these maps
existing_maps = gis.content.search(query=f"title:'{map_title}'", item_type = "Web Map")

if existing_maps:
    # retrieve the existing web map item
    web_map_item = existing_maps[0]
    web_map = WebMap(web_map_item)
    print ("Map already exists within AGOL Content")

else:
    print("No existing map found")

In [ ]:
#manage map creation depending on whether or not the map already exists

    """
    Updates an existing web map or creates a new one in ArcGIS Online (AGOL) based 
    on user input.

    This script performs the following actions:
    1. If a web map with the specified title exists:
       - Prompts the user to decide whether to update the map.
       - If the user chooses to update, it adds new layers to the map if they are 
       not already present, updates the web map, and provides a URL to access the 
       updated map.
       - If the user does not wish to update, it provides a URL to access the existing 
       map without changes.
    2. If no web map with the specified title exists:
       - Creates a new web map, adds predefined layers to it, and saves it with 
       specified properties.
       - Provides a URL to access the newly created web map.

    Args:
        existing_maps (list): List of web map items retrieved from AGOL search.
        live_bike_fl (FeatureLayer): The live bike feature layer to add to the map.
        BikeStationsZone_data (FeatureLayer): The bike stations/zone feature layer to add to the map.
        web_map_item (Item): The web map item to update or use as a reference.
        map_title (str): Title for the new web map if one needs to be created.

    Returns:
        None

    Raises:
        ImportError: If the display functionality is not available in the environment.
    """

if existing_maps:
    # Ask user if they want to update the map
    user_input = input("Do you wish to update the map? (Y/N): ").strip().upper()
    
    if user_input == 'Y':
        # update the map with layers defined but first check if layers are already in the map before adding
        layers_to_add = [live_bike_fl, BikeStationsZone_data]
        for layer in layers_to_add:
            if not any(existing_layer.id == layer.id for existing_layer in web_map.layers):
                web_map.add_layer(layer)
                
        web_map.update()
        web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
        print(f"Webmap updated successfully. Access the map here: {web_map_url}")
        display(web_map)
        
    else:
        # user has no updates so construct a URL to access the map
        web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
        print(f"No updates required. Access the existing map here: {web_map_url}")
        display(web_map)

else: 
    # if map doesn't exist, create one
    web_map = WebMap()
    web_map.add_layer(layer=live_bike_fl)
    web_map.add_layer(BikeStationsZone_data)
    web_map_properties = {
        "title": map_title,
        "description": "A web map of live Dublin Bike locations and their zones/stations obtained from Smart Dublin API",
        "tags": "Dublin Bike, locations, live webmap",
        "snippet": "A webmap of Dublin Bike locations"
    }
    
    web_map_item = web_map.save(item_properties=web_map_properties)
    web_map_url = f"https://www.arcgis.com/home/webmap/viewer.html?webmap={web_map_item.id}"
    print(f"New web map created and saved. Access it here: {web_map_url}")
    
    try:
        display(web_map)

    except ImportError:
        print("Display functionality not available in this environment.")